In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import model_zoo

import data_freiburg_numpy_to_hdf5

In [ ]:


# ======================================================================
debug_dir = "/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/logdir/unet3d_da_0.0nchannels1_rrun_small_image_set/results/visualization/"

In [ ]:
list_files_train = os.listdir(debug_dir + 'training/')
image_number_train = 0
list_files_train

In [ ]:
image_filter_train = list(filter(lambda x: f'_image_{image_number_train}' in x, list_files_train))
image_filter_train

In [ ]:
step = 400
pred_train = np.load(os.path.join(debug_dir,'training', list(filter(lambda x: f'_{step}' in x, image_filter_train))[0]))
gt_train = np.load(os.path.join(debug_dir,'training', list(filter(lambda x: f'_{step}' in x, image_filter_train))[1]))
input_train = np.load(os.path.join(debug_dir,'training', list(filter(lambda x: f'_{step}' in x, image_filter_train))[2]))
np.where(gt_train ==1), np.where(pred_train ==1)


In [ ]:
pred_train.shape, gt_train.shape, input_train.shape

In [ ]:
def plot_batches(array, input_image = False):

    if input_image:
        fig, axs = plt.subplots(2, 4, figsize=(10,10))
        for n, ax in enumerate(axs.reshape(-1)):
            ax.imshow(array[n,0,:,:,3])
        plt.show()
    else:
        fig, axs = plt.subplots(2, 4, figsize=(10,10))
        for n, ax in enumerate(axs.reshape(-1)):
            ax.imshow(array[n,:,:,3])
        plt.show()

In [ ]:
plot_batches(pred_train)

In [ ]:
plot_batches(gt_train)

In [ ]:
plot_batches(input_train, input_image= True)

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))
batch_image = 1
for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(pred_train[batch_image, :, :, i])


In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))

for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(gt_train[batch_image, :, :, i])


In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))

for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(input_train[batch_image, 1,:, :, i])


In [ ]:
# load saved model
loss = "dice"
out_channels = 2
in_channels = 4
model_path = f'/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/logdir/unet3d_da_0.25nchannels{in_channels}_rclean_03_{loss}'
best_model_path = os.path.join(model_path, list(filter(lambda x: 'best' in x, os.listdir(model_path)))[-1])

model = model_zoo.UNet(in_channels, out_channels)
model.load_state_dict(torch.load(best_model_path, map_location=torch.device('cpu')))
model.eval()

In [ ]:
project_data_root = '/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg'
data_vl = data_freiburg_numpy_to_hdf5.load_data(basepath = project_data_root,

                                                        idx_start = 20,
                                                        idx_end = 24,
                                                        train_test='validation')
images_vl = data_vl['images_validation']
labels_vl = data_vl['labels_validation']        

In [ ]:
images_vl.shape, labels_vl.shape

In [ ]:
# Data augmentation dimension 
# Set trian inside the loop 

In [ ]:
batch_size = 8
np.random.seed(0)
inputs = images_vl[np.sort(np.random.randint(len(images_vl), size = batch_size))]
np.random.seed(0)
labels = labels_vl[np.sort(np.random.randint(len(images_vl), size = batch_size))]

inputs = torch.from_numpy(inputs)
# Input (batch_size, channell,x,y,z)
inputs.transpose_(1,4).transpose_(2,4).transpose_(3,4)
labels = torch.from_numpy(labels)

In [ ]:
pred = model(inputs)

In [ ]:
import torch.nn.functional as F
prediction = F.softmax(pred, dim=1).argmax(dim = 1)

In [ ]:
prediction.shape, labels.shape

In [ ]:
%matplotlib inline
def plot_time(t = 0):
    fig, axs = plt.subplots(4,4);
    img = 0
    for i, ax in enumerate(axs.reshape(-1)):
        
        if i%2 == 0:
            ax.imshow(prediction[img,:,:,t].detach().numpy())
            

        else:
            ax.imshow(labels[img,:,:,t].detach().numpy())   
            img = img + 1 
    fig.show()

In [ ]:
plot_time(0)

In [ ]:
plot_time(1)

In [ ]:
plot_time(20)

In [ ]:
results_dir = '/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/logdir/unet3d_da_0.0nchannels4_rdebug_entropy_one_patient/results/visualization/training'
list_files = os.listdir(results_dir)
list_files

In [ ]:
image_number = 0
image_filter = list(filter(lambda x: f'_image_{image_number}' in x, list_files))

In [ ]:
epoch = 540
pred = np.load(os.path.join(results_dir, list(filter(lambda x: f'step_{epoch}' in x, image_filter))[0]))
gt = np.load(os.path.join(results_dir, list(filter(lambda x: f'step_{epoch}' in x, image_filter))[1]))
np.where(gt ==1), np.where(pred ==1)


In [ ]:
pred.shape

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))

for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(pred[7, :, :, i])


In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))

for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(gt[7, :, :, i])


In [ ]:

plt.imshow(pred[4, :, :, 10], cmap='gray'), 
#plt.imshow(gt[0, :, :, 0], cmap='gray')

In [ ]:
results_dir_debug = '/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/logdir/unet3d_da_0.0nchannels4_rsingle_image_cross_1000_direct_cross_loss/results/visualization/'

In [ ]:
list_files_train = os.listdir(results_dir_debug + 'training/')
image_number_train = 0
image_filter_train = list(filter(lambda x: f'_image_{image_number_train}' in x, list_files_train))

In [ ]:
step = 600
pred_train = np.load(os.path.join(results_dir_debug,'training', list(filter(lambda x: f'_{step}' in x, list_files_train))[0]))
gt_train = np.load(os.path.join(results_dir_debug,'training', list(filter(lambda x: f'_{step}' in x, list_files_train))[1]))
np.where(gt_train ==1), np.where(pred_train ==1)


In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))

for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(pred_train[0, :, :, i])


In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10, 10))

for i, ax in enumerate(axs.reshape(-1)):
    
    ax.imshow(gt_train[0, :, :, i])


In [ ]:
(gt_train[0,...,15] == gt_train[0,...,40]).all()

In [ ]:
# Shitty test to check batches 

path = "/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/testing_batches/batch1"

In [ ]:
img = np.load(os.path.join(path, 'images.npy'))
seg = np.load(os.path.join(path, 'labels.npy'))

In [ ]:
img.shape, seg.shape

In [ ]:
plot_batches(img)

In [ ]:
plot_batches(seg)

In [ ]:
# Different main debug
path_data_debug_main = "/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/testing_batches/cut_channel_1_debug_01/"

In [ ]:
os.listdir(path_data_debug_main)

In [ ]:
n = 120
from scipy.special import softmax


img = np.load(os.path.join(path_data_debug_main, f'images_{n}.npy'))

seg = np.load(os.path.join(path_data_debug_main, f'labels_{n}.npy'))

pred = np.load(os.path.join(path_data_debug_main, f'prediction_{n}.npy'))

img.shape, seg.shape, pred.shape

In [ ]:
plot_batches(img, input_image= True)

In [ ]:
plot_batches(seg)

In [ ]:
plot_batches(softmax(pred, axis = 1).argmax(axis = 1))

In [ ]:
seg.shape, softmax(pred, axis = 1).argmax(axis = 1).shape

In [ ]:
%matplotlib inline
def plot_time_(t = 0):
    fig, axs = plt.subplots(4,4);
    img = 0
    for i, ax in enumerate(axs.reshape(-1)):
        
        if i%2 == 0:
            ax.imshow(softmax(pred, axis = 1).argmax(axis = 1)[img,:,:,t])
            

        else:
            ax.imshow(seg[img,:,:,t])   
            img = img + 1 
    fig.show()

In [ ]:
plot_time_(0)

In [ ]:
# Check single image ouput
path_single_image = "/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/testing_batches/cut_channel_1_debug_01/"


In [ ]:
plt.imshow(in_[2, :, :, 1,1])

In [ ]:
plt.imshow(out_[2, :, :, 1])

In [ ]:



path_cut_2 = "/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/logdir/unet3d_da_0.0nchannels4_r_phase_dice_cut_z_2_debug_print_input/"

input_cut = np.load(path_cut_2 + 'inputs_train.npy')

In [ ]:

label_cut = np.load(path_cut_2 + 'labels_train.npy')

In [ ]:
input_cut.shape
slice_ = 0

In [ ]:
%matplotlib inline
plt.imshow(input_cut[slice_, :,:, 3,1])
plt.imshow(label_cut[slice_, :,:, 3], alpha= 0.1)
slice_ +=1 
plt.show()

In [ ]:
project_data_root = '/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg'
data_tr = data_freiburg_numpy_to_hdf5.load_data(basepath = project_data_root,
                                                    idx_start = 0,
                                                    idx_end = 19,
                                                    train_test='train')
images_tr = data_tr['images_train']
labels_tr = data_tr['labels_train']

In [ ]:
#slice_ = 0
%matplotlib inline
plt.imshow(images_tr[slice_, :,:, 3,1])
plt.show()
plt.imshow(images_tr[slice_, :,:, 3,0])
plt.imshow(labels_tr[slice_, :,:, 3], alpha= 0.1)
slice_ +=1 
plt.show()